In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import collections
import folium
import re

In [2]:
!pip install haversine
from haversine import haversine #위도,경도를 이용해 거리계산

In [3]:
total = pd.read_csv('/content/drive/MyDrive/dataset/project_team3_delivery/yogiyo_delivery4.csv')

In [4]:
total = total[['name','categories','restaurant_type', 'review_avg', 'lat', 'lng', 'phone', 'address']] # 121804개의 데이터
total.dropna(subset=['address'], inplace=True) #주소가 없는 음식점 제외
total = total[total['address'].str.startswith('서울')] #주소가 서울인곳만

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
def foo(x):
  return x[2:-2]

def bar(x):
  s = ''
  for i in x:
    s = s +i + ' '
  return s[:-1]

In [6]:
total['categories'] = total['categories'].apply(foo)
total['categories'] = total.categories.str.split('\', \'').apply(bar)
# 카테고리별로 공백으로 구분한 문자열로 치환

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [7]:
total

,name,categories,restaurant_type,review_avg,lat,lng,phone,address
10980,깻잎두마리치킨-중랑점,프랜차이즈 치킨 야식,food,4.7,37.574935,127.085789,5.071861e+10,서울 중랑구 면목동 371-122 한성빌딩 1층
18008,참치의명가&야식포차,회초밥 테이크아웃 찜탕 야식 일식돈까스,food,4.4,37.488103,126.907341,5.071863e+10,서울특별시 동작구 신대방동 686-2 686-2 1층
27864,가락시장보은수산,일식돈까스 테이크아웃 야식 한식,food,4.7,37.496117,127.111032,5.035288e+10,서울 송파구 가락동 600 가락동 농수산물도매시장 비동 지하1층 4-1호
28008,가락시장온달수산,1인분주문 일식돈까스,food,5.0,37.496106,127.111017,5.035372e+10,"서울 송파구 가락동 600 가락동 농수산물도매시장 가락몰판매동 수산부류 1층94호,..."
28739,한우투플육회&생연어회,한식 야식 일식돈까스,food,2.6,37.520785,126.853678,5.071296e+10,"서울 양천구 신정동 1030-6 210호,211호"
...,...,...,...,...,...,...,...,...
113033,GS25-상계우리점,편의점 프랜차이즈,grocery,5.0,37.666594,127.069710,5.070278e+10,서울 노원구 상계동 135-14
113034,교촌치킨-상계역점,치킨 프랜차이즈,food,4.5,37.662067,127.075537,5.035260e+10,서울특별시 노원구 중계동 137-39 정진아트리움 137-39 정진아트리움 102호 전체
113035,BBQ-상계보람점,치킨 테이크아웃 프랜차이즈,food,4.7,37.662518,127.069679,5.035299e+10,서울 노원구 상계동 169-367 1층
113793,요거프레소-도봉산점,프랜차이즈 카페디저트,food,0.0,37.687212,127.044233,5.035298e+10,서울 도봉구 도봉동 556-19


In [8]:
total[total['categories'].str.contains('헬스뷰티')].head(20)


,name,categories,restaurant_type,review_avg,lat,lng,phone,address
44104,아리따움-금천남문시장점,헬스뷰티 프랜차이즈,grocery,0.0,37.472390,126.901581,NaN,서울특별시 금천구 독산동 165-5 1층 가운데점포
44602,아리따움-난곡점,테이크아웃 프랜차이즈 헬스뷰티,grocery,5.0,37.470786,126.920062,NaN,서울특별시 관악구 신림동 635-23 1층
51146,랄라블라-신림점,프랜차이즈 헬스뷰티,grocery,5.0,37.482916,126.929571,5.035285e+10,서울 관악구 신림동 1638-19 1층
51154,랄라블라-구로디지털점,헬스뷰티 프랜차이즈,grocery,5.0,37.483418,126.900975,5.035285e+10,서울 구로구 구로동 1125-6
51467,아리따움-난곡사거리점,헬스뷰티 테이크아웃 프랜차이즈,grocery,5.0,37.481258,126.914810,NaN,서울특별시 관악구 신림동 1569-4 이솝빌딩
52092,아리따움-관악중앙점,프랜차이즈 헬스뷰티 테이크아웃,grocery,4.9,37.480948,126.952084,NaN,"서울특별시 관악구 봉천동 862-1 에그엘로우 1층 8,9,10,11호"
53635,아리따움-아리따움양재역점,프랜차이즈 헬스뷰티,grocery,0.0,37.484088,127.037756,NaN,서울특별시 서초구 양재동 11-84 1층
55688,아리따움-송파점,테이크아웃 헬스뷰티 프랜차이즈,grocery,4.9,37.503612,127.113358,5.035371e+10,서울특별시 송파구 송파동 104-4 우암빌딩 104-4 우암빌딩 104
59402,아리따움-개봉2동점,프랜차이즈 헬스뷰티 테이크아웃,grocery,4.9,37.491744,126.855638,NaN,서울특별시 구로구 개봉동 403-51 1층
60082,아리따움-구로디지털단지역점,헬스뷰티 테이크아웃 프랜차이즈,grocery,4.9,37.483392,126.901424,NaN,서울특별시 구로구 구로동 1125-5 동연빌딩 1층


In [9]:
categories_all = []
for i in total.index:
  a = total.loc[i,'categories'].split()
  categories_all = list(set(categories_all + a))
categories_all #['다회용기', '한식', '반려동물용품', '프랜차이즈', '1인분주문', '고기구이', '샌드위치', '피자양식', '샐러드', '치킨', '카페디저트', '편의점', '도시락죽', '일식돈까스', '아시안', '중식', '버거', '분식', '찜탕', '회초밥', '헬스뷰티', '테이크아웃', '족발보쌈', '야식', '신규맛집']

['프랜차이즈',
 '치킨',
 '피자양식',
 '아시안',
 '반려동물용품',
 '샌드위치',
 '찜탕',
 '분식',
 '편의점',
 '야식',
 '다회용기',
 '샐러드',
 '테이크아웃',
 '회초밥',
 '버거',
 '신규맛집',
 '헬스뷰티',
 '일식돈까스',
 '고기구이',
 '카페디저트',
 '한식',
 '1인분주문',
 '중식',
 '도시락죽',
 '족발보쌈']

In [10]:
total = total[~total['categories'].str.contains('반려동물용품')]
total = total[~total['categories'].str.contains('헬스뷰티')]
total = total[~total['categories'].str.contains('다회용기')]
total = total[~total['restaurant_type'].str.contains('grocery')]
total.apply(lambda x: x.str.strip(), axis = 1)
# total.drop(["distance"], axis=1)
total = total.reset_index(drop=True)
total

,name,categories,restaurant_type,review_avg,lat,lng,phone,address
0,깻잎두마리치킨-중랑점,프랜차이즈 치킨 야식,food,4.7,37.574935,127.085789,5.071861e+10,서울 중랑구 면목동 371-122 한성빌딩 1층
1,참치의명가&야식포차,회초밥 테이크아웃 찜탕 야식 일식돈까스,food,4.4,37.488103,126.907341,5.071863e+10,서울특별시 동작구 신대방동 686-2 686-2 1층
2,가락시장보은수산,일식돈까스 테이크아웃 야식 한식,food,4.7,37.496117,127.111032,5.035288e+10,서울 송파구 가락동 600 가락동 농수산물도매시장 비동 지하1층 4-1호
3,가락시장온달수산,1인분주문 일식돈까스,food,5.0,37.496106,127.111017,5.035372e+10,"서울 송파구 가락동 600 가락동 농수산물도매시장 가락몰판매동 수산부류 1층94호,..."
4,한우투플육회&생연어회,한식 야식 일식돈까스,food,2.6,37.520785,126.853678,5.071296e+10,"서울 양천구 신정동 1030-6 210호,211호"
...,...,...,...,...,...,...,...,...
45422,한우정육식당,한식,food,0.0,37.668002,127.079127,5.035286e+10,서울 노원구 상계동 101-177
45423,교촌치킨-상계역점,치킨 프랜차이즈,food,4.5,37.662067,127.075537,5.035260e+10,서울특별시 노원구 중계동 137-39 정진아트리움 137-39 정진아트리움 102호 전체
45424,BBQ-상계보람점,치킨 테이크아웃 프랜차이즈,food,4.7,37.662518,127.069679,5.035299e+10,서울 노원구 상계동 169-367 1층
45425,요거프레소-도봉산점,프랜차이즈 카페디저트,food,0.0,37.687212,127.044233,5.035298e+10,서울 도봉구 도봉동 556-19


In [11]:
# me = (37.557945, 126.99419)
# tmp = total
# tmp['distance'] = total.apply(lambda row: haversine(me, (row['lat'], row['lng'])), axis=1)
# tmp = total[total['categories'].isin(['한식','일식돈까스']) & (tmp['distance'] <= 2)].head(10)
# map_osm = folium.Map(location=[tmp.iloc[0]['lat'], tmp.iloc[0]['lng']], zoom_start=13)
# folium.Marker(me, popup=' 현재위치', icon = folium.Icon('red', icon='star')).add_to(map_osm)
# for region in range(len(tmp)):
#     # CircleMarker를 사용하여 지역마다 원형마커를 생성합니다.
#     folium.Marker([tmp.iloc[region]['lat'], tmp.iloc[region]['lng']],
#               popup=tmp.iloc[region]['name'],
#               icon=folium.Icon('blue', icon='star'),
#              ).add_to(map_osm)

# map_osm

In [12]:
total[total.categories.str.contains('버거')]
# new_categories=['분식', '야식', '족발/보쌈', '찜탕', '치킨', '카페/디저트', '피자', '한식', '회', '돈까스/일식', '패스트푸드', '아시안/양식', '도시락', '중식']


,name,categories,restaurant_type,review_avg,lat,lng,phone,address
37,BURGER&BEER-금천점,1인분주문 야식 피자양식 버거 치킨,food,4.4,37.454385,126.901803,5.071862e+10,서울 금천구 시흥동 884-4 2층
38,BURGERSTAGE-금천점,야식 1인분주문 피자양식 버거 치킨,food,4.5,37.454385,126.901803,5.071862e+10,서울 금천구 시흥동 884-4 2층
258,롯데리아-시흥중앙점,버거 치킨 프랜차이즈 야식 1인분주문 신규맛집,food,4.8,37.450836,126.908830,5.070287e+10,"서울특별시 금천구 시흥동 911-2 지상1,2층"
364,마법주방-반포직영점,피자양식 1인분주문 야식 테이크아웃 치킨 버거,food,4.2,37.508261,127.021627,5.071290e+10,서울 서초구 반포동 724-14 102호
403,봉구스밥버거-양재점,버거 한식 테이크아웃 프랜차이즈 분식,food,4.6,37.472076,127.046413,5.035285e+10,서울특별시 서초구 양재동 391-18 창성빌딩 104호
...,...,...,...,...,...,...,...,...
45279,맘스터치-노원로데오점,버거 치킨 프랜차이즈 신규맛집,food,4.8,37.656981,127.062485,5.037257e+10,서울특별시 노원구 상계동 593-2 동원빌딩 2층
45286,맥도날드-중계점,1인분주문 프랜차이즈 버거,food,4.6,37.650853,127.076021,5.071297e+10,서울 노원구 중계동 359-14 원광빌딩 1층
45297,버거킹-하계점,프랜차이즈 버거 신규맛집,food,4.6,37.639101,127.075113,NaN,서울특별시 노원구 하계동 256-8 삼화빌딩 1층
45345,맥도날드-방학역DT점,버거 프랜차이즈 1인분주문,food,4.6,37.670109,127.043323,5.071300e+10,"서울 도봉구 도봉동 620-24 지상1,2층"


In [20]:
for i in total.index:
  c = total.loc[i,'categories']
  c = c.replace('족발보쌈', '족발/보쌈')
  c = c.replace('카페디저트', '카페/디저트')
  c = c.replace('회초밥', '회')
  c = c.replace('일식돈까스', '일식')
  c = c.replace('아시안', '아시안/양식')
  c = c.replace('도시락죽', '도시락')
  if '피자양식' in c:
    c = c.replace('아시안', '')
    c = c.replace('피자양식','피자 아시안/양식')
  else:
    c = c.replace('아시안', '아시안/양식')
  total.loc[i,'categories'] = c
  k = total.loc[i,'name']
  k = k.replace(',','.')
  total.loc[i,'name'] = k

In [14]:
#total[total.categories.str.contains('1인분주문')]
total.drop(['restaurant_type'], axis=1, inplace=True)

In [15]:
# for i in total.index:
#   c = total.loc[i,'name']
#   c = c.replace('\'', '\\\'')
#   total.loc[i,'name'] = c
len(total[total.name.str.contains(',')])
# total.to_csv("yogiyo delivery in seoul.csv", index=False, encoding='utf-8-sig')
# total.head()

0

In [16]:
total['phone'] = total['phone'].fillna(-1)
total['phone'] = total['phone'].apply(int)
total['phone'] = '0' + total['phone'].apply(str)
total['phone'] = total['phone'].replace('0-1', np.nan)

total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45427 entries, 0 to 45426
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        45427 non-null  object 
 1   categories  45427 non-null  object 
 2   review_avg  45427 non-null  float64
 3   lat         45427 non-null  float64
 4   lng         45427 non-null  float64
 5   phone       40588 non-null  object 
 6   address     45427 non-null  object 
dtypes: float64(3), object(4)
memory usage: 2.4+ MB


In [17]:
for i in total.index:
  a = total.loc[i,'name']
  reg = re.compile(r'[가-힣]+n[가-힣]+')
  if reg.match(a):
    a = re.sub('n','&', a)
    total.loc[i,'name'] = a


In [18]:
total.loc[992,'name'] = '바로삶은보양족발' # 바로삶은보양족발n1인세트
total.loc[5072,'name'] = '삼두삼계탕&아구찜' # 삼두삼계탕and아구찜
total.loc[41255,'name'] = '황학동 곱창&막창' # 황학동곱창and막창
# total.loc[34964,'name'] = '국수and전골전문' # 국수and전골전문
total.loc[40607,'name'] = '좋은날엔' # 1인커피전문점좋은날n카페

In [19]:
total.to_csv("yogiyo delivery in seoul.csv", index=False, encoding='utf-8-sig')